In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate,PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain


llm = OpenAI(model_name="gpt-3.5-turbo-instruct")
chat = ChatOpenAI(model_name="gpt-3.5-turbo")

# First Chain: Generates a Haiku about a given programming language
haiku_prompt = ChatPromptTemplate.from_template(
    "Write a Haiku about the programming language {language}. It should capture its essence and characteristics."
)
haiku_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo"), prompt=haiku_prompt)

# Second Chain: Explains the Haiku
explanation_prompt = ChatPromptTemplate.from_template(
    "Explain the meaning of the following Haiku about a programming language:\n\n{haiku}\n\nProvide an insightful explanation."
)
explanation_chain = LLMChain(llm=ChatOpenAI(model="gpt-3.5-turbo"), prompt=explanation_prompt)

# Chain them together
final_chain = SimpleSequentialChain(chains=[haiku_chain, explanation_chain], verbose=True)

final_chain.run("javascript")



> Entering new SimpleSequentialChain chain...
Dynamic and fast
JavaScript weaves magic code
Web's heart beats with it
This Haiku is describing JavaScript, a popular programming language used for building dynamic and interactive websites. The first line, "Dynamic and fast," highlights the dynamic nature of JavaScript, which allows for real-time changes and updates on web pages. The second line, "JavaScript weaves magic code," suggests that JavaScript has the ability to create powerful and engaging features on websites. Finally, the third line, "Web's heart beats with it," emphasizes the importance of JavaScript in the functioning and overall user experience of the web. Overall, this Haiku captures the essence of JavaScript as a versatile and essential tool for web development.

> Finished chain.


'This Haiku is describing JavaScript, a popular programming language used for building dynamic and interactive websites. The first line, "Dynamic and fast," highlights the dynamic nature of JavaScript, which allows for real-time changes and updates on web pages. The second line, "JavaScript weaves magic code," suggests that JavaScript has the ability to create powerful and engaging features on websites. Finally, the third line, "Web\'s heart beats with it," emphasizes the importance of JavaScript in the functioning and overall user experience of the web. Overall, this Haiku captures the essence of JavaScript as a versatile and essential tool for web development.'

In [23]:

from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate

# ✅ 예제 데이터 (Few-Shot Learning)
examples = [
    {
        "movie": "인셉션",
        "response": (
            "**제목:** 인셉션\n"
            "**감독:** 크리스토퍼 놀란\n"
            "**주연:** 레오나르도 디카프리오, 조셉 고든 레빗, 엘리엇 페이지, 톰 하디\n"
            "**예산:** 1억 6천만 달러\n"
            "**박스오피스 수익:** 8억 2,990만 달러\n"
            "**장르:** SF, 스릴러\n"
            "**시놉시스:** 꿈속에서 정보를 훔치는 특수 요원 '돔 코브'는 기억을 지울 수 있는 새로운 임무를 맡게 된다. 그러나 꿈의 세계에서 현실과 환상이 뒤섞이며 예상치 못한 위기가 닥친다."
        ),
    },
    {
        "movie": "대부",
        "response": (
            "**제목:** 대부\n"
            "**감독:** 프란시스 포드 코폴라\n"
            "**주연:** 말론 브란도, 알 파치노, 제임스 칸, 다이안 키튼\n"
            "**예산:** 600~720만 달러\n"
            "**박스오피스 수익:** 2억 5천만~2억 8천 7백만 달러\n"
            "**장르:** 범죄, 드라마\n"
            "**시놉시스:** 뉴욕 마피아의 대부 비토 코를레오네는 조직을 이끌어가지만, 암살 시도 이후 아들 마이클 코를레오네가 가업을 물려받으며 변화가 찾아온다."
        ),
    }
]

# ✅ 예제 출력 형식을 위한 프롬프트 템플릿
example_prompt = PromptTemplate.from_template(
    "영화 제목: {movie}\n응답:\n{response}"
)

# ✅ FewShotPromptTemplate 생성
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="당신은 영화 데이터베이스 전문가입니다. 주어진 영화 제목에 대해 항상 동일한 형식으로 정보를 제공합니다.",
    suffix="영화 제목: {movie}\n응답:",
    input_variables=["movie"]
)

# ✅ LLM 체인 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
movie_chain = LLMChain(llm=llm, prompt=few_shot_prompt)

# ✅ 영화 정보를 가져오는 함수
def get_movie_details(movie_name):
    return movie_chain.run({"movie": movie_name})

# ✅ 실행 예제
movie_name = "가나의 혼인잔치"  # 원하는 영화 제목으로 변경 가능
result = get_movie_details(movie_name)

print(result)

**제목:** 가나의 혼인잔치
**감독:** 조지 클루니
**주연:** 조지 클루니, 애미 아담스, 존 트라볼타, 크리스토퍼 워컨
**예산:** 알려지지 않음
**박스오피스 수익:** 3,100만 달러
**장르:** 코미디, 드라마
**시놉시스:** 1950년대 미국, 빈곤한 가나와 그의 아내 로즈는 부자 가족의 혼인식에 초대받게 된다. 그러나 가난한 가나와 로즈가 혼인식에 참석하게 되면서 예상치 못한 사건이 벌어지게 된다.


In [41]:
from langchain.memory import ConversationBufferMemory

# ✅ Few-Shot 예제 데이터 (항상 3개의 이모지를 반환하도록 유도)
examples = [
    {"movie": "Top Gun", "emojis": "🛩️👨‍✈️🔥"},
    {"movie": "The Godfather", "emojis": "👨‍👨‍👦🔫🍝"},
    {"movie": "Finding Nemo", "emojis": "🐠🌊🦈"},
    {"movie": "Titanic", "emojis": "🚢❤️💔"},
    {"movie": "Harry Potter", "emojis": "⚡📖🧙"},
]

# ✅ FewShotPromptTemplate 설정
example_prompt = PromptTemplate(
    input_variables=["movie", "emojis"],
    template="Movie: {movie}\n Emojis: {emojis}"
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a movie expert. Based on the movie title, provide exactly 3 emojis that best represent it.\n\nPrevious History:\n{chat_history}\n\n",
    suffix="Movie: {movie}\nEmojis:",
    input_variables=["movie"]
)

# ✅ Memory 설정 (메모리 유지)
memory = ConversationBufferMemory(memory_key="chat_history",max_token_limit=100 , return_messages=True,k=5)

# ✅ LLM 체인 생성
llm = ChatOpenAI(model_name="gpt-3.5-turbo")
emoji_chain = LLMChain(llm=llm, prompt=few_shot_prompt, memory=memory)

# ✅ 체인 실행 함수
def get_movie_emojis(movie_name):
    return emoji_chain.run({"movie": movie_name})



In [42]:
# ✅ 첫 번째 영화 질문
first_movie = "Avatar"
first_result = get_movie_emojis(first_movie)
print(f"First movie ({first_movie}): {first_result}")

# ✅ 두 번째 영화 질문
second_movie = "The Matrix"
second_result = get_movie_emojis(second_movie)
print(f"Second movie ({second_movie}): {second_result}")

# ✅ 세 번째 영화 질문
third_movie = "Jurassic Park"
third_result = get_movie_emojis(third_movie)
print(f"Third movie ({third_movie}): {third_result}")

# ✅ 네 번째 영화 질문 (이제 첫 번째 영화는 메모리에서 사라짐)
fourth_movie = "Star Wars"
fourth_result = get_movie_emojis(fourth_movie)
print(f"Fourth movie ({fourth_movie}): {fourth_result}")



First movie (Avatar): 🌎💙👽
Second movie (The Matrix): 🕶️💻🔵
Third movie (Jurassic Park): 🦕🌴🔬
Fourth movie (Star Wars): 🌌⚔️🤖


In [43]:
# ✅ 이전에 질문한 영화 기억하는지 확인
previous_result = get_movie_emojis("Avatar")
print("result", previous_result)
previous_question = emoji_chain.memory.load_memory_variables({})
print("Memory Recall:", previous_question)

result 🌎💙👽
Memory Recall: {'chat_history': [HumanMessage(content='Avatar'), AIMessage(content='🌎💙👽'), HumanMessage(content='The Matrix'), AIMessage(content='🕶️💻🔵'), HumanMessage(content='Jurassic Park'), AIMessage(content='🦕🌴🔬'), HumanMessage(content='Star Wars'), AIMessage(content='🌌⚔️🤖'), HumanMessage(content='Avatar'), AIMessage(content='🌎💙👽')]}


In [ ]:
from langchain.document_loaders import UnstructuredFileLoader
import requests
from langchain.docstore.document import Document
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.qa_with_sources.stuff_prompt import PROMPT
from langchain.prompts import PromptTemplate
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory

# ✅ 1. 문서 가져오기 (from Gist)
url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw"
response = requests.get(url)
text = response.text

# ✅ 2. 문서를 LangChain 문서 형태로 변환
docs = [Document(page_content=text)]

# ✅ 3. 문서 분할 (StuffDocuments는 전체 텍스트를 그대로 사용 가능하지만 RAG 효율을 위해 나눕니다)
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)

# ✅ 4. 벡터 저장소 구축 (Embedding + Vectorstore)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding)

# ✅ 5. Retriever 설정
retriever = vectorstore.as_retriever()

# ✅ 6. Memory 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# ✅ 7. StuffDocuments Chain 수동 구성
from langchain.chains.retrieval_qa.base import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo"),
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT},
    memory=memory,
    return_source_documents=True
)


Created a chunk of size 1444, which is longer than the specified 1000
Created a chunk of size 1251, which is longer than the specified 1000
Created a chunk of size 1012, which is longer than the specified 1000
Created a chunk of size 2313, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 1673, which is longer than the specified 1000
Created a chunk of size 1137, which is longer than the specified 1000
Created a chunk of size 1559, which is longer than the specified 1000
Created a chunk of size 1200, which is longer than the specified 1000
Created a chunk of size 1042, which is longer than the specified 1000
Created a chunk of size 1345, which is longer than the specified 1000
Created a chunk of size 1339, which is longer than the specified 1000
Created a chunk of size 1288, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of s

ValidationError: 1 validation error for StuffDocumentsChain
__root__
  document_variable_name context was not found in llm_chain input_variables: ['question', 'summaries'] (type=value_error)